<h1 align='center'> 영상처리 프로그래밍 실습 4</h1>

<h6 align='right'> 2025. 4. 3. </h6>

<div class="alert alert-block alert-info">
    
- 파일 이름에서 00000000을 자신의 학번으로, name을 자신의 이름으로 수정하세요.

- 다음 줄에 자신의 이름, 학번, 학과(전공)을 적으세요.

* 이름: 이선재  &nbsp;&nbsp;          학번:  20227123  &nbsp;&nbsp;         학과(전공): 빅데이터
    
</div>

- JupyterLab 문서의 최신 버전은 [JupyterLab Documentation](https://jupyterlab.readthedocs.io/en/stable/index.html#/)을  참고하라

- Markdown은 [Markdown Guide](https://www.markdownguide.org/)를 참고하라.
- [Markdown Cheat Sheet](https://www.markdownguide.org/cheat-sheet/)

* 제출 마감: 4월 9일 (수) 오후 10:00까지 최종본을 SmartLEAD제출


In [2]:
import cv2
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import sys

print("Python version", sys.version)
print("OpenCV version", cv2.__version__)
print("NumPy version", np.__version__)

Python version 3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]
OpenCV version 4.11.0
NumPy version 1.26.4



## 5주차 정리

### 1. Main Body에서 글로벌 변수와 로컬 변수
- **설명**:
  - `main` 블록(최상위 스코프)에서 선언된 변수는 기본적으로 **글로벌 변수**입니다.
  - 하지만 함수 내에서 동일한 이름의 변수를 새로 정의하면, 이는 **로컬 변수**로 간주됩니다. 예:
    ```python
    img = "global image"  # 글로벌 변수
    def load_image():
        img = "local image"  # 로컬 변수, 글로벌 변수와 별개
        print(img)
    load_image()  # 출력: "local image"
    print(img)    # 출력: "global image"
    ```
  - 함수에서 글로벌 변수를 사용하려면 `global` 키워드를 명시적으로 선언해야 합니다:
    ```python
    img = "global image"
    def load_image():
        global img  # 글로벌 변수 사용 선언
        img = "modified image"
    load_image()
    print(img)  # 출력: "modified image"
    ```
- **주의**:
  - 함수에서 변수를 새로 지정하지 않고 사용하면, 파이썬은 이를 글로벌 변수로 간주합니다(읽기 전용). 하지만 수정하려면 `global` 선언이 필요합니다.

---

### 2. `raise`는 글로벌에서 작용되는가?
- **설명**:
  - `raise`는 파이썬에서 예외를 발생시키는 명령어로, **호출 스택(call stack)을 통해 상위 스코프**로 예외를 전파합니다.
  - 함수 내부에서 `raise`가 실행되면:
    1. 해당 함수가 즉시 종료되고,
    2. 예외가 호출한 상위 스코프(예: `try-except`)로 전달됩니다.
  - **"글로벌에서 작용된다"는 표현**:
    - `raise` 자체는 로컬에서 실행되지만, 예외가 처리되지 않으면 호출 스택을 따라 최종적으로 **글로벌 스코프**(프로그램의 메인 레벨)까지 도달할 수 있습니다.
    - 예: `try-except`가 없으면 프로그램이 종료되며, 이는 글로벌 스코프에 영향을 미친다고 볼 수 있습니다.
  - **코드에서의 동작**:
    ```python
    def load_image(filename):
        if not os.path.exists(filename):
            raise FileNotFoundError(f"File {filename} does not exist")
    try:
        load_image("bird.png")
    except FileNotFoundError as e:
        print(f"오류: {e}")  # 예외가 여기서 처리됨
    ```
    - 이 경우 `try-except`가 예외를 잡으므로 글로벌 스코프까지 전파되지 않습니다.
- **결론**:
  - `raise`는 로컬에서 실행되지만, 예외는 상위로 전파된다.
  - `try-except`가 없으면 글로벌 스코프에서 프로그램 종료로 이어질 수 있다.

---

### 3. 화살표(`->`)와 콜론(`:`)의 역할
- **설명**:
  - 파이썬에서 `->`와 `:`는 **타입 힌트(type hint)**를 제공합니다. 코드의 가독성을 높이고, 개발 도구(예: IDE)에서 타입 검사를 지원합니다.
  - 예:
    ```python
    def load_image(filename: str) -> np.ndarray:
        return cv2.imread(filename)
    ```
    - `filename: str`: `filename`은 문자열(`str`) 타입이어야 함을 힌트.
    - `-> np.ndarray`: 함수가 반환하는 값은 NumPy 배열(`np.ndarray`)임을 힌트.
- **특징**:
  - 타입 힌트는 실행 시 강제되지 않으며, 단지 문서화 및 개발 도구 지원용입니다.

---

### 4. DocString: 함수 설명서
- **설명**:
  - **DocString**은 함수나 클래스에 대한 설명을 제공하는 문자열로, `help()` 함수 호출 시 표시됩니다.
  - 일반적으로 함수 정의 바로 아래에 `"""`로 작성합니다:
    ```python
    def load_image(filename: str) -> np.ndarray:
        """파일에서 이미지를 로드합니다.

        Args:
            filename (str): 이미지 파일 경로
        Returns:
            np.ndarray: 로드된 이미지 데이터
        Raises:
            FileNotFoundError: 파일이 존재하지 않을 경우
        """
        return cv2.imread(filename)
    ```
  - 실행: `help(load_image)`로 확인 가능.

---

### 5. 자주 사용하는 함수를 라이브러리 모듈로 설정
- **설명**:
  - 자주 사용하는 함수는 별도의 모듈(파일)로 만들어 재사용성을 높일 수 있습니다.
  - 예: `image_tools.py`라는 파일에 함수를 정의:
    ```python
    # image_tools.py
    import cv2
    def load_image(filename: str) -> np.ndarray:
        if not os.path.exists(filename):
            raise FileNotFoundError(f"File {filename} does not exist")
        return cv2.imread(filename)
    ```
  - 사용:
    ```python
    # main.py
    from utils import load_image
    img = load_image("bird.png")
    ```
- **질문: "라이브러리 안의 import를 모두 가져와야 하나?"**
  - **답변**: 아니요. 필요한 함수나 클래스만 선택적으로 `import`하면 됩니다.
    - `from utils import load_image`: `load_image`만 가져옴.
    - `import utils`: 전체 모듈을 가져오고, `utils.load_image()`로 사용.
  - 모듈 내부에서 `import cv2`를 했더라도, 사용하는 파일에서 다시 `import cv2`를 할 필요는 없습니다(단, `cv2`를 직접 사용하지 않는 한).

---

### 6. 상수는 대문자로 설정
- **설명**:
  - 파이썬에서 상수(constant)는 관례적으로 **대문자**로 작성합니다. (파이썬은 진정한 상수를 강제하지 않지만, 개발자 간 약속입니다.)
  - 예:
    ```python
    IMAGE_FILE = "bird.png"
    WINDOW_NAME = "Blending"
    ```
  - 상수는 값이 변경되지 않음을 나타내며, 코드 가독성을 높입니다.

---



----
추가되는 내용

### 이미지를 가지고 뭔가를 하기전에 복사하는 이유

### mouse_callback 함수의 event 동작 원리?(요기는 조금 자세하게 설명해줘.)

### cv2.namedWindow(WINDOW_CROPPED, 0)  # 0과 cv2.WINDOW_NORMAL 의 차이가 뭐야

### start_x, start_y = -1, -1 의 동작 원리가 뭐야

### cropped = param[y1:y2+1, x1:x2+1]
----

In [1]:
# 직접 만든 라이브러리를 import 합니다.
import image_tools

# 상수는 대문자로 설정합니다.
IMAGE_FILE = 'bird.png'
WINDOW_MAIN = "Image"

## 문제 1. 
 - 목표: 타입 힌트, DocString, 모듈 이해하기

### 1.1 기본 영상 표시와 종료

'bird.png' 파일을 읽고 윈도우에 표시하며, 'q'로 종료하는 프로그램을 작성하라.
단, 지난 실습 시간에 만들었던 load_image 함수를 사용하고, main body에서는 예외 처리를 해야 한다.

In [ ]:
# 함수안에서 global img를 했을 때, 
def load_image():
    global img
    print(img.dtype)
    img = 1

img = cv2.imread('bird.png')
load_image()
print(img)

uint8
1


In [ ]:
def load_image(filename: str) -> np.ndarray:
    if not os.path.exists(filename):
        # raise는 함수안에서 사용을 한다고 하여도 main body에서 작용되는 글로벌로 작용된다?
        raise FileNotFoundError(f"File {filename} does not exist")
    img: np.ndarray = cv2.imread(filename)
    if img is None:
        return ValueError(f"Could Not load {filename}")
    return img

try:
    # main에 있는 변수이지만, 함수 load_image에서 사용되고 있는 img라는 변수는 다른 변수입니다.
    # 하지만 이렇게 하는것이 아닌 먼저 변수를 지정하고 함수를 사용한다면 그리고 함수안에서 그 변수를 따로 지정하지 않는다면
    # 로컬 변수가 되는게 아닌 글로벌 변수로 사용할 수 있습니다.
    img = load_image('bird.png')

    cv2.namedWindow("Blending", cv2.WINDOW_AUTOSIZE)

    while True:
        cv2.imshow("Blending", img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except FileNotFoundError as e:
    # 파일 관련 오류 처리
    print(f"오류: {e}")
except ValueError as e:
    print(f"Error : {e}")
finally:
    cv2.destroyAllWindows()

### 1.2 type hint 사용하기

1. 타입 힌트 개요
- Python의 타입 힌트(Type Hint)는 코드에서 변수, 함수 매개변수, 반환값 등의 데이터 타입을 명시할 수 있게 해주는 기능
- Python 3.5부터 도입
- 주로 코드의 가독성을 높이고, 개발 도구(예: IDE, 정적 분석 도구)를 활용해 타입 오류를 사전에 잡아내는 데 사용됨

2. 타입 힌트란?
- Python은 기본적으로 동적 타입 언어로, 변수 선언 시 타입을 명시하지 않아도 됨
- 그러나 타입 힌트를 사용하면 코드에 "이 변수나 함수는 어떤 타입을 기대한다"는 정보를 추가할 수 있음
- 목적
  * 코드 문서화: 타입을 명시해 읽기 쉽게 만듦.
  * 오류 방지: 개발 중 타입 불일치를 미리 감지.
  * 도구 지원: PyCharm, VSCode 같은 IDE에서 자동 완성과 타입 체크 제공.
- 특징
  * 타입 힌트는 실행 시 강제되지 않음 (즉, 런타임에 영향을 주지 않음).
  * 단지 힌트일 뿐이며, 정적 타입 검사 도구(예: mypy)로만 강제할 수 있음

3. 기본 사용법
- 타입 힌트는 변수 선언, 함수 매개변수, 반환값에 적용할 수 있음.

4. 기본 문법

(1) 변수에 타입 힌트

```python
x: int = 10
name: str = "Alice"
numbers: list = [1, 2, 3]
```
 - 뒤에 타입을 명시.
 - 초기화 값은 선택 사항(없어도 됨: x: int).
(2) 함수에 타입 힌트

```python
def add(a: int, b: int) -> int:
    return a + b
```
 - 매개변수: a: int, b: int로 타입 지정.
 - 반환값: -> int로 반환 타입 지정.

(3) 복잡한 타입
typing 모듈을 사용해 더 구체적인 타입을 표현할 수 있음

```python
from typing import List, Dict, Optional

def greet(names: List[str]) -> str:
    return ", ".join(names)

def get_score(student: Dict[str, int]) -> Optional[int]:
    return student.get("score")  # None 가능
```

 - List[str]: 문자열 리스트.
 - Dict[str, int]: 키는 문자열, 값은 정수인 딕셔너리.
 - Optional[int]: int 또는 None을 반환.

In [ ]:
# filename에서 : str 와 -> np.ndarray 는
# 이 자료를 사용한다는 힌트 같은것 입니다.
def load_image(filename: str) -> np.ndarray:
    if not os.path.exists(filename):
        # raise는 함수안에서 사용을 한다고 하여도 main body에서 작용되는 글로벌로 작용된다?
        raise FileNotFoundError(f"File {filename} does not exist")
    img: np.ndarray = cv2.imread(filename)
    if img is None:
        return ValueError(f"Could Not load {filename}")
    return img

### 1.3 함수의 DocString 이해하기

In [ ]:
help(min)

Help on built-in function min in module builtins:

min(...)
    min(iterable, *[, default=obj, key=func]) -> value
    min(arg1, arg2, *args, *[, key=func]) -> value

    With a single iterable argument, return its smallest item. The
    default keyword-only argument specifies an object to return if
    the provided iterable is empty.
    With two or more arguments, return the smallest argument.



In [12]:
def load_image(filename: str) -> np.ndarray:
    """영상 파일을 읽고 래더잴ㄷ"""

In [13]:
help(load_image)

Help on function load_image in module __main__:

load_image(filename: str) -> numpy.ndarray
    영상 파일을 읽고 래더잴ㄷ



### 1.4 load_image 함수를 모듈의 일부로 만들기
- 모듈의 이름 정하기: image_tools
- load_image 함수 부분만 image_tools.py 파일로 저장한다.

In [6]:
import image_tools

# 상수는 대문자로 설정합니다.
IMAGE_FILE = 'bird.png'
WINDOW_MAIN = "Image"

try:
    # 라이브러리 안의 모듈을 사용하여 편하게 함수를 사용하는 모습입니다.
    img = image_tools.load_image(IMAGE_FILE)

    cv2.namedWindow(WINDOW_MAIN, cv2.WINDOW_AUTOSIZE)

    while True:
        cv2.imshow(WINDOW_MAIN, img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except FileNotFoundError as e:
    # 파일 관련 오류 처리
    print(f"오류: {e}")
except ValueError as e:
    print(f"Error : {e}")
finally:
    cv2.destroyAllWindows()

## 문제 2. 
다음 조건을 모두 만족하는 프로그램을 작성하려고 한다. 아래 지시에 따라 단계적으로 프로그램을 작성하라.

1. 'bird.png' 파일을 읽고, cv2.imshow 함수를 사용하여 "Image"라는 제목의 윈도우에 영상을 표시한다.
2. cv2.namedWindow("Cropped", 0)을 사용하여 "Cropped"라는 제목의 크기 조절 가능한 윈도우를 생성한다.
3. "Image" 윈도우에서 마우스 왼쪽 버튼을 누른 상태로 마우스를 움직이면, 처음 누른 위치를 사각형의 한 꼭지점으로 하고 현재 마우스 위치를 대각선 반대쪽 꼭지점으로 하는 사각형이 실시간으로 그려진다.
4. 마우스를 움직이는 동안 사각형의 크기가 동적으로 변한다.
5. 마우스 왼쪽 버튼을 놓으면(릴리스 시), 그 순간 사각형으로 정의된 영역의 영상이 "Cropped" 윈도우에 표시된다.
6. 새로운 위치에서 마우스 왼쪽 버튼을 다시 누르면 3번과 4번 동작이 반복된다.
7. 키보드에서 'q'를 누르면 모든 윈도우가 닫히고 프로그램이 종료된다.

**힌트**
1. cv2.EVENT_LBUTTONDOWN, MOUSEMOVE, LBUTTONUP 이벤트를 사용해야 함.
2. 좌표를 저장하려면 전역 변수나 클래스 활용 가능.
3. 영상 경계를 벗어나면 오류가 발생하니 min, max로 범위 조정 필요.

### 2.1 [단계 1] 마우스 이벤트 연결과 클릭 감지

문제 1에서 작성한 프로그램을 확장하여, 마우스 이벤트를 처리하는 콜백 함수를 추가하라. 이 함수는 왼쪽 마우스 버튼이 눌릴 때(cv2.EVENT_LBUTTONDOWN) 클릭된 위치의 좌표(x, y)를 콘솔에 출력해야 한다. 콜백 함수의 이름은 mouse_callback로 하고, 함수 시그니처는 아래를 따라야 한다.

```python
mouse_callback(event, x, y, flags, param)
```

프로그램이 실행되면 창에 영상이 표시되고, 마우스 클릭 시 콘솔에 좌표가 출력되도록 한다.

In [ ]:
import cv2
import numpy as np
import os

# 상수 정의
IMAGE_FILE = "bird.png"
WINDOW_IMAGE = "Image"
WINDOW_CROPPED = "Cropped"

# 글로벌 변수 (마우스 이벤트 좌표 저장)
drawing = False  # 드래그 중인지 여부
start_x, start_y = -1, -1  # 시작 좌표

def mouse_callback(event: int, x: int, y: int, flags: int, param: np.ndarray) -> None:
    """마우스 이벤트를 처리하여 클릭 좌표를 출력하고 사각형을 관리합니다.

    Args:
        event (int): 마우스 이벤트 타입
        x (int): 마우스 x 좌표
        y (int): 마우스 y 좌표
        flags (int): 추가 플래그
        param (np.ndarray): 원본 이미지 데이터
    """
    # 함수안에서 메인바디의 변수를 사용하기 위해서 global를 지정
    global drawing, start_x, start_y

    # 원본 이미지 복사
    img = param.copy()

    # 마우스 왼쪽 버튼 누름 (클릭 감지)
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        start_x, start_y = x, y
        print(f"Clicked at: ({x}, {y})")  # 클릭 좌표 출력

    # 마우스 이동 (드래그 중)
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            # 사각형 그리기 (실시간 업데이트)
            cv2.rectangle(img, (start_x, start_y), (x, y), (0, 255, 0), 2)
            cv2.imshow(WINDOW_IMAGE, img)

    # 마우스 왼쪽 버튼 릴리스
    elif event == cv2.EVENT_LBUTTONUP:
        if drawing:
            drawing = False
            # 사각형 좌표 계산 (음수 방지 및 경계 조정)
            x1, x2 = min(start_x, x), max(start_x, x)
            y1, y2 = min(start_y, y), max(start_y, y)
            h, w = param.shape[:2]
            x1, x2 = max(0, x1), min(w - 1, x2)
            y1, y2 = max(0, y1), min(h - 1, y2)

            # 크롭된 이미지 추출 및 표시
            cropped = param[y1:y2, x1:x2]
            if cropped.size > 0:  # 크롭 영역이 유효한 경우
                cv2.imshow(WINDOW_CROPPED, cropped)

try:
    # 이미지 로드
    img = image_tools.load_image(IMAGE_FILE)
    # 원본 이미지 복사본은 mouse_callback 함수에서 처리

    # 윈도우 생성
    cv2.namedWindow(WINDOW_IMAGE)
    # cv2.namedWindow(WINDOW_CROPPED, cv2.WINDOW_NORMAL)  # 크기 조절 가능
    cv2.namedWindow(WINDOW_CROPPED, 0)  # 0과 cv2.WINDOW_NORMAL 의 차이가 뭐야

    # 마우스 콜백 설정
    cv2.setMouseCallback(WINDOW_IMAGE, mouse_callback, img)

    # 초기 이미지 표시
    cv2.imshow(WINDOW_IMAGE, img)

    # 'q' 키로 종료
    while True:
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except (FileNotFoundError, ValueError) as e:
    print(f"오류: {e}")
# except 
finally:
    cv2.destroyAllWindows()

Clicked at: (232, 302)
Clicked at: (428, 95)


### 2.2 OpenCV에서 이미지 좌표계 이해하기
- 프로그램을 실행하라.
- 마우스 왼쪽 버튼으로 이미지의 네 모서리(왼쪽 위, 오른쪽 위, 왼쪽 아래, 오른쪽 아래)를 각각 클릭하라.
- 콘솔에 출력되는 좌표로 부터 아래 표에 들어갈 좌표를 추정해서 적어라. (정확한 모서리를 클릭하는 것이 어려울 수 있으니, 영상 데이터를 표현하는 배열의 shape을 활용하라.)


|모서리 | 좌표 (x, y) |
--|--
|왼쪽 위	| |
|오른쪽 위	 | |
|왼쪽 아래	 | |
|오른쪽 아래 | |

In [15]:
print(img[0].shape())

TypeError: 'tuple' object is not callable

### 2.3 mouse_callback 함수의 파라미터 이해

다음 함수에서 각 파라미터들에 대해서 설명하라.

```
mouse_callback(event, x, y, flags, param)
```

---

#### `mouse_callback` 함수 파라미터 설명

##### 1. `event` (int)
- **설명**: 마우스 이벤트의 종류를 나타내는 정수 값입니다. OpenCV에서 정의된 상수로, 발생한 마우스 동작을 식별합니다.
- **주요 값**:
  - `cv2.EVENT_LBUTTONDOWN`: 마우스 왼쪽 버튼이 눌렸을 때.
  - `cv2.EVENT_LBUTTONUP`: 마우스 왼쪽 버튼이 떼어졌을 때.
  - `cv2.EVENT_MOUSEMOVE`: 마우스가 이동할 때.
  - `cv2.EVENT_RBUTTONDOWN`: 마우스 오른쪽 버튼이 눌렸을 때.
  - `cv2.EVENT_MBUTTONDOWN`: 마우스 가운데 버튼(휠 버튼)이 눌렸을 때.
  - 기타: `EVENT_LBUTTONDBLCLK` (왼쪽 버튼 더블 클릭) 등.
- **용도**: 특정 마우스 동작(예: 클릭, 드래그 시작/종료)을 감지하고 그에 맞는 로직을 실행합니다.

##### 2. `x` (int)
- **설명**: 마우스 이벤트가 발생한 위치의 **x 좌표** (수평 위치)입니다. 이미지의 왼쪽 상단이 `(0, 0)` 기준으로 픽셀 단위로 측정됩니다.
- **용도**: 클릭 또는 이동한 위치를 파악하거나, 사각형의 시작/끝 좌표로 활용할 수 있습니다.

##### 3. `y` (int)
- **설명**: 마우스 이벤트가 발생한 위치의 **y 좌표** (수직 위치)입니다. 이미지의 왼쪽 상단이 `(0, 0)` 기준으로 픽셀 단위로 측정됩니다.
- **용도**: `x`와 함께 좌표 쌍 `(x, y)`를 형성하여 정확한 위치를 지정합니다.

##### 4. `flags` (int)
- **설명**: 마우스 이벤트와 함께 발생한 추가 상태를 나타내는 플래그 값입니다. 여러 상태가 비트 연산으로 조합될 수 있습니다.
- **주요 값**:
  - `cv2.EVENT_FLAG_CTRLKEY`: Ctrl 키가 눌린 상태.
  - `cv2.EVENT_FLAG_SHIFTKEY`: Shift 키가 눌린 상태.
  - `cv2.EVENT_FLAG_ALTKEY`: Alt 키가 눌린 상태.
  - `cv2.EVENT_FLAG_LBUTTON`: 왼쪽 버튼이 눌린 상태 (드래그 중 확인 가능).
- **용도**: 키보드와 마우스 동작을 조합한 조건을 처리할 때 사용. 예: Ctrl + 클릭 감지.

##### 5. `param` (object)
- **설명**: `cv2.setMouseCallback()` 호출 시 사용자 정의로 전달된 추가 데이터입니다. 타입은 제한이 없으며, 콜백 함수에 넘겨줄 객체를 지정할 수 있습니다.
- **용도**: 주로 원본 이미지(`np.ndarray`)나 기타 필요한 데이터를 전달하여 콜백 함수 내에서 활용합니다.
- **예시**: 이전 코드에서 `param`으로 원본 이미지(`img`)를 전달하여 사각형 그리기 및 크롭에 사용.

---

#### 예제에서의 사용
```python
def mouse_callback(event: int, x: int, y: int, flags: int, param: np.ndarray) -> None:
    if event == cv2.EVENT_LBUTTONDOWN:
        print(f"Clicked at: ({x}, {y})")  # 클릭 좌표 출력
    elif event == cv2.EVENT_MOUSEMOVE and flags & cv2.EVENT_FLAG_LBUTTON:
        print(f"Dragging at: ({x}, {y})")  # 드래그 중 좌표 출력
```

- `event`: `EVENT_LBUTTONDOWN`으로 클릭 감지, `EVENT_MOUSEMOVE`로 이동 감지.
- `x, y`: 클릭 또는 이동 위치 출력.
- `flags`: 드래그 중인지 확인 (`EVENT_FLAG_LBUTTON`).
- `param`: 이미지 데이터로 활용 가능 (여기서는 사용 안 함).

---

#### 요약
| 파라미터 | 타입   | 설명                              | 주요 용도                  |
|----------|--------|-----------------------------------|----------------------------|
| `event`  | `int`  | 마우스 이벤트 종류                | 이벤트 타입 감지           |
| `x`      | `int`  | 마우스 x 좌표                    | 위치 지정                  |
| `y`      | `int`  | 마우스 y 좌표                    | 위치 지정                  |
| `flags`  | `int`  | 추가 상태 플래그                 | 키보드/마우스 상태 확인    |
| `param`  | `object` | 사용자 정의 전달 데이터          | 추가 데이터 활용           |

이 파라미터들은 OpenCV의 마우스 이벤트 처리에서 필수적인 역할을 하며, 다양한 상호작용을 구현할 수 있게 합니다. 추가 질문이 있으면 말씀해주세요!

### 2.4 [단계 2] 드래그 시작과 상태 관리

문제 2.1에서 만든 프로그램을 다음과 같이 수정하라.
- drawing 변수를 만들고 False로 초기화한다. 만약에 기존에 화면에 사각형이 있다면 사각형을 지운다.
- 왼쪽 마우스 클릭 시 그 위치를 저장한다.
- 왼쪽 마우스를 클릭한 상태에서 마우스를 drawing을 True로 설정하고 사각형을 지속적으로 업데이트한다
- 클릭한 왼쪽 마우스를 놓으면 drawing을 False로 하고 사각형 업데이트를 중단하고 최종 사각형을 표시한다.
- mouse_callback 함수의 이름을 draw_rectangle로 수정하라.

In [ ]:
import cv2
import numpy as np
import os

# 상수 정의
IMAGE_FILE = "bird.png"
WINDOW_IMAGE = "Image"
WINDOW_CROPPED = "Cropped"

# 글로벌 변수 (마우스 이벤트 좌표 저장)
drawing = False  # 드래그 중인지 여부
start_x, start_y = -1, -1  # 시작 좌표

def draw_rectangle(event: int, x: int, y: int, flags: int, param: np.ndarray) -> None:
    """마우스 이벤트를 처리하여 클릭 좌표를 출력하고 사각형을 관리합니다.

    Args:
        event (int): 마우스 이벤트 타입
        x (int): 마우스 x 좌표
        y (int): 마우스 y 좌표
        flags (int): 추가 플래그
        param (np.ndarray): 원본 이미지 데이터
    """
    # 함수안에서 메인바디의 변수를 사용하기 위해서 global를 지정
    global drawing, start_x, start_y

    # 원본 이미지 복사
    img = param.copy()

    # 마우스 왼쪽 버튼 누름 (클릭 감지)
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        start_x, start_y = x, y
        # img_copy = img.copy() # 원본 복사본 초기화
        print(f"Clicked at: ({x}, {y})")  # 클릭 좌표 출력

    # 마우스 이동 (드래그 중)
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            # img_copy = img.copy()
            # 사각형 그리기 (실시간 업데이트)
            cv2.rectangle(img, (start_x, start_y), (x, y), (0, 255, 0), 2)
            cv2.imshow(WINDOW_IMAGE, img)

    # 마우스 왼쪽 버튼 릴리스
    elif event == cv2.EVENT_LBUTTONUP:
        if drawing:
            drawing = False
            # 사각형 좌표 계산 (음수 방지 및 경계 조정)
            x1, x2 = min(start_x, x), max(start_x, x)
            y1, y2 = min(start_y, y), max(start_y, y)
            h, w = param.shape[:2]
            x1, x2 = max(0, x1), min(w - 1, x2)
            y1, y2 = max(0, y1), min(h - 1, y2)

            # 크롭된 이미지 추출 및 표시(x와 y가 정상적으로 할려면 이렇게 해야됨)
            cropped = param[y1:y2+1, x1:x2+1]
            if cropped.size > 0:  # 크롭 영역이 유효한 경우
                cv2.imshow(WINDOW_CROPPED, cropped)

try:
    # 이미지 로드
    img = image_tools.load_image(IMAGE_FILE)
    # 원본 이미지 복사본은 draw_rectangle 함수에서 처리

    # 윈도우 생성
    cv2.namedWindow(WINDOW_IMAGE)
    # cv2.namedWindow(WINDOW_CROPPED, cv2.WINDOW_NORMAL)  # 크기 조절 가능
    cv2.namedWindow(WINDOW_CROPPED, 0)  # 0과 cv2.WINDOW_NORMAL 의 차이가 뭐야

    # 마우스 콜백 설정
    cv2.setMouseCallback(WINDOW_IMAGE, draw_rectangle, img)

    # 초기 이미지 표시
    cv2.imshow(WINDOW_IMAGE, img)

    # 'q' 키로 종료
    while True:
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except (FileNotFoundError, ValueError) as e:
    print(f"오류: {e}")
# except 
finally:
    cv2.destroyAllWindows()

Clicked at: (285, 235)
Clicked at: (547, 43)
Clicked at: (181, 329)
Clicked at: (364, 163)
Clicked at: (364, 163)
Clicked at: (405, 198)
Clicked at: (464, 240)
Clicked at: (240, 26)


### 2.4 [단계 3] 최종 프로그램
문제 2에서 설명한 모든 기능을 완성하라.

In [ ]:
import cv2
import numpy as np
import os

# 상수 정의
IMAGE_FILE = "bird.png"
WINDOW_IMAGE = "Image"
WINDOW_CROPPED = "Cropped"

# 글로벌 변수 (마우스 이벤트 좌표 저장)
drawing = False  # 드래그 중인지 여부
start_x, start_y = -1, -1  # 시작 좌표

def draw_rectangle(event: int, x: int, y: int, flags: int, param: np.ndarray) -> None:
    """마우스 이벤트를 처리하여 클릭 좌표를 출력하고 사각형을 관리합니다.

    Args:
        event (int): 마우스 이벤트 타입
        x (int): 마우스 x 좌표
        y (int): 마우스 y 좌표
        flags (int): 추가 플래그
        param (np.ndarray): 원본 이미지 데이터
    """
    # 함수안에서 메인바디의 변수를 사용하기 위해서 global를 지정
    global drawing, start_x, start_y

    # 원본 이미지 복사
    img = param.copy()

    # 마우스 왼쪽 버튼 누름 (클릭 감지)
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        start_x, start_y = x, y
        # img_copy = img.copy() # 원본 복사본 초기화
        print(f"Clicked at: ({x}, {y})")  # 클릭 좌표 출력

    # 마우스 이동 (드래그 중)
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            # img_copy = img.copy()
            # 사각형 그리기 (실시간 업데이트)
            cv2.rectangle(img, (start_x, start_y), (x, y), (0, 255, 0), 2)
            cv2.imshow(WINDOW_IMAGE, img)

    # 마우스 왼쪽 버튼 릴리스
    elif event == cv2.EVENT_LBUTTONUP:
        if drawing:
            drawing = False
            # 사각형 좌표 계산 (음수 방지 및 경계 조정)
            x1, x2 = min(start_x, x), max(start_x, x)
            y1, y2 = min(start_y, y), max(start_y, y)
            h, w = param.shape[:2]
            x1, x2 = max(0, x1), min(w - 1, x2)
            y1, y2 = max(0, y1), min(h - 1, y2)

            # 크롭된 이미지 추출 및 표시(x와 y가 정상적으로 할려면 이렇게 해야됨)
            cropped = param[y1:y2+1, x1:x2+1]
            if cropped.size > 0:  # 크롭 영역이 유효한 경우
                cv2.imshow(WINDOW_CROPPED, cropped)

try:
    # 이미지 로드
    img = image_tools.load_image(IMAGE_FILE)
    # 원본 이미지 복사본은 draw_rectangle 함수에서 처리

    # 윈도우 생성
    cv2.namedWindow(WINDOW_IMAGE)
    cv2.namedWindow(WINDOW_CROPPED, cv2.WINDOW_NORMAL)  # 크기 조절 가능
    # cv2.namedWindow(WINDOW_CROPPED, )  # 0과 cv2.WINDOW_NORMAL 의 차이가 뭐야

    # 마우스 콜백 설정
    cv2.setMouseCallback(WINDOW_IMAGE, draw_rectangle, img)

    # 초기 이미지 표시
    cv2.imshow(WINDOW_IMAGE, img)

    # 'q' 키로 종료
    while True:
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except (FileNotFoundError, ValueError) as e:
    print(f"오류: {e}")
# except 
finally:
    cv2.destroyAllWindows()

### 2.5 프로그램 실행 과정 이해

이 프로그램이 실행되는 과정을 설명하라.


### 2.6 콜백 함수 params에 딕셔너리를 전달하는 방법

In [ ]:
# 딕셔너리의 키는 왜 str으로 처리하는 걸까?
# 뭐였지 hash?
state = {
    'img' : img,
    'img_copy' : img.copy(),
    'drawing' : False,
    'start_x' : -1,
    'start_y' : -1
}

In [ ]:
import cv2
import numpy as np
import os

# 상수 정의
IMAGE_FILE = "bird.png"
WINDOW_IMAGE = "Image"
WINDOW_CROPPED = "Cropped"

state = {
    'img' : img,
    'img_copy' : img.copy(),
    'drawing' : False,
    'start_x' : -1,
    'start_y' : -1
}

def draw_rectangle(event: int, x: int, y: int, flags: int, param: np.ndarray) -> None:
    """마우스 이벤트를 처리하여 클릭 좌표를 출력하고 사각형을 관리합니다.

    Args:
        event (int): 마우스 이벤트 타입
        x (int): 마우스 x 좌표
        y (int): 마우스 y 좌표
        flags (int): 추가 플래그
        param (np.ndarray): 원본 이미지 데이터
    """
    # 딕셔너리
    state = param

    # 원본 이미지 복사
    img = param.copy()

    # 마우스 왼쪽 버튼 누름 (클릭 감지)
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        start_x, start_y = x, y
        # img_copy = img.copy() # 원본 복사본 초기화
        print(f"Clicked at: ({x}, {y})")  # 클릭 좌표 출력

    # 마우스 이동 (드래그 중)
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            # img_copy = img.copy()
            # 사각형 그리기 (실시간 업데이트)
            cv2.rectangle(img, (start_x, start_y), (x, y), (0, 255, 0), 2)
            cv2.imshow(WINDOW_IMAGE, img)

    # 마우스 왼쪽 버튼 릴리스
    elif event == cv2.EVENT_LBUTTONUP:
        if drawing:
            drawing = False
            # 사각형 좌표 계산 (음수 방지 및 경계 조정)
            x1, x2 = min(start_x, x), max(start_x, x)
            y1, y2 = min(start_y, y), max(start_y, y)
            h, w = param.shape[:2]
            x1, x2 = max(0, x1), min(w - 1, x2)
            y1, y2 = max(0, y1), min(h - 1, y2)

            # 크롭된 이미지 추출 및 표시(x와 y가 정상적으로 할려면 이렇게 해야됨)
            cropped = param[y1:y2+1, x1:x2+1]
            if cropped.size > 0:  # 크롭 영역이 유효한 경우
                cv2.imshow(WINDOW_CROPPED, cropped)

try:
    # 이미지 로드
    img = image_tools.load_image(IMAGE_FILE)
    # 원본 이미지 복사본은 draw_rectangle 함수에서 처리

    # 윈도우 생성
    cv2.namedWindow(WINDOW_IMAGE)
    # cv2.namedWindow(WINDOW_CROPPED, cv2.WINDOW_NORMAL)  # 크기 조절 가능
    cv2.namedWindow(WINDOW_CROPPED, 0)  # 0과 cv2.WINDOW_NORMAL 의 차이가 뭐야

    # 마우스 콜백 설정
    cv2.setMouseCallback(WINDOW_IMAGE, draw_rectangle, state)

    # 초기 이미지 표시
    cv2.imshow(WINDOW_IMAGE, img)

    # 'q' 키로 종료
    while True:
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except (FileNotFoundError, ValueError) as e:
    print(f"오류: {e}")
# except 
finally:
    cv2.destroyAllWindows()

Clicked at: (292, 359)


error: OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'rectangle'
> Overload resolution failed:
>  - img is not a numpy array, neither a scalar
>  - img is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'img'
>  - Expected Ptr<cv::UMat> for argument 'img'


error: OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'rectangle'
> Overload resolution failed:
>  - img is not a numpy array, neither a scalar
>  - img is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'img'
>  - Expected Ptr<cv::UMat> for argument 'img'


error: OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'rectangle'
> Overload resolution failed:
>  - img is not a numpy array, neither a scalar
>  - img is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'img'
>  - Expected Ptr<cv::UMat> for argument 'img'


error: OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'rectangle'
> Overload resolution failed:
>  - img is not a numpy array, neither a scalar
>  - img is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'img'
>  - Expected Ptr<cv::UMat> for argument 'img'


error: OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'rectangle'
> Overload resolution failed:
>  - img is not a numpy array, neither a scalar
>  - img is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'img'
>  - Expected Ptr<cv::UMat> for argument 'img'


error: OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'rectangle'
> Overload resolution failed:
>  - img is not a numpy array, neither a scalar
>  - img is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'img'
>  - Expected Ptr<cv::UMat> for argument 'img'


error: OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'rectangle'
> Overload resolution failed:
>  - img is not a numpy array, neither a scalar
>  - img is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'img'
>  - Expected Ptr<cv::UMat> for argument 'img'


AttributeError: 'dict' object has no attribute 'shape'

### 2.7 클래스를 이용하는 방법

In [ ]:
class ImageDrawer:
    def __init__(self, img):
        self.img = img
        self.img_copy = img.copy()
        self.drawing = False
        self.start_x = -1
        self.start_y = -1

    def draw_rectangle(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBU
        
try:
    img = image_tools.load_image(IMAGE_FILE)
    drawer = ImageDrawer(img)
    
    cv2.namedWindow(WINDOW_IMAGE)
    cv2.namedWindow(WINDOW_CROPPED)  # 크기 조절 가능

## 문제 3. NumPy ndarray 객체 구조 이해

### 3.1 NumPy 데이터 타입 이해
```
getsizeof(...)
    getsizeof(object [, default]) -> int
    
    Return the size of object in bytes.
```

### 3.2. 
sys.getsizeof() 함수를 사용하여 ndarray의 원소의 개수에 따른 객체의 크기 확인하기


#### 3.2.1 ndarray의 dtype에 따른 ndarray 객체의 크기 살펴 보기

#### 3.2.2.
sys.getsizeof() 함수를 사용하여 ndarray의 원소의 개수에 따른 객체의 크기 확인하기


In [ ]:
for n in range(5):
    a = np.array([k for k in range(n)], dtype=np.uint8)
    show_array_info(a)

In [ ]:
for n in range(5):
    a = np.array([k for k in range(n)], dtype=np.uint64)
    show_array_info(a)

#### 3.2.3 
sys.getsizeof() 함수를 사용하여 ndarray와 view 객체의 크기 확인하기
